In [1]:
import os
import time
import pandas as pd
import numpy as np
import sqlalchemy
import psycopg2
import sys
print(pd.__version__)

from postgressql import *

1.3.3


In [2]:
conn_params_dic = {
    "host": "10.0.0.102",
    "dbname": "mastr",
    "user": "uba_user",
    "password": "UBAit2021!",
    "port": "5432"
}

sqlpath = "./sqlCommands/loadMastrDBfiles.sql"

def format_bytes(size):
    # 2**10 = 1024
    power = 2**10
    n = 0
    power_labels = {0 : '', 1: 'kilo', 2: 'mega', 3: 'giga', 4: 'tera'}
    while size > power:
        size /= power
        n += 1
    return size, power_labels[n]+'bytes'

def build_postgres_conn_string (param:dict) -> str:
    """

    """
    return f'postgresql+psycopg2://{param["user"]}:{param["password"]}@{param["host"]}:{param["port"]}/{param["dbname"]}'

def create_postgres_engine (param:dict):
    """
    
    """
    conString = build_postgres_conn_string(param)
    engine = sqlalchemy.create_engine(conString, pool_recycle=3600)
    return engine

def read_postgres_from_queryfile (sqlpath:str, postgresLogin:dict):# -> pandas.core.frame.DataFrame:
    """

    """
    engine = create_postgres_engine(postgresLogin)
    
    scriptFile = open(sqlpath,'r')
    script = scriptFile.read()
    df = pd.read_sql(script, engine)

    return df

import updateMastr

# LOOP THROUGH ALL XML FILES

In [ ]:
stackedList = updateMastr.list_xml_files()

In [ ]:
filterList = ["Netzanschlusspunkte"] # More can be added...

print(len(stackedList))

for i, liste in enumerate(stackedList):
    stackedList[i] = [x for x in liste if all(y not in x for y in filterList)]
stackedList = [x for x in stackedList if x != []]

print(len(stackedList))

# CREATE TABLE based on Pandas DF

In [ ]:
engine = create_postgres_engine(conn_params_dic)

for files in stackedList:
    tableName = files[0].split("/")[-1].split(".")[0].split("_")[0]

    print(f"{tableName} start reading into dataFrame")
    ListDfs = [pd.read_xml(path_or_buffer=file, encoding="utf-16") for file in files]

    print(f"{tableName} start concating dataFrames")
    df = pd.concat(ListDfs,ignore_index=True)
    print("Changing dtypes to dateTime.")
    change_Dtype_Datetime(df)

    print(f"{tableName} load dataFrames into Postgres-DB")
    df.to_sql(
        name=tableName,
        schema="mastr_raw",
        con=engine,
        if_exists="replace",
        index=False
    )
    del(df, ListDfs, tableName)
    print(".")


In [16]:
engine = create_postgres_engine(conn_params_dic)

# Creating Points from Läng und Breit columns
sqlQuery = open("sqlCommands/addPointColumnFromKoord.sql")
escaped_sql = sqlalchemy.text(sqlQuery.read())
engine.execute(escaped_sql)